In [1]:
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
from IPython.display import IFrame
import folium 
from sklearn.preprocessing import MinMaxScaler
from folium.plugins import HeatMap
from collections import defaultdict
import branca.colormap

### Definitions for all scenarios

In [6]:
def calculate_POI_density_for_each_link(this_df,this_node_csv,hotspot_column_name, number_of_hotspot_users_column_name):
    df_copy = this_df.copy()
    
    link_csv_POIdata = np.zeros(len(df_copy.index))
    link_csv_integerindex = np.arange(len(df_copy))
    link_csv_index = np.array(df_copy.index)
    traffic_signals = np.zeros(len(df_copy.index))
    hotspots = np.zeros(len(df_copy.index))
    hotspot_users = np.zeros(len(df_copy.index))
    for i in link_csv_integerindex:
        item = link_csv_index[i]
        link_index_v = df_copy.loc[item,'v']
        link_index_u = df_copy.loc[item,'u']
        # Put POI Density information
        POI_data = this_node_csv.loc[link_index_v,'Closest Poi Density_with distance decay']
        POI_data_2 = this_node_csv.loc[link_index_u,'Closest Poi Density_with distance decay']
        # Each link is connected to 2 nodes. We find the POI density for each node and find the mean of the two
        # Alternatively, we could use the max of the two
        mean_POI = (POI_data+POI_data_2)/2
        link_csv_POIdata[i] = mean_POI

        # Put traffic signal information
        if (this_node_csv.loc[link_index_v,'highway']=='traffic_signals')|(this_node_csv.loc[link_index_u,'highway']=='traffic_signals'):
            traffic_signals[i]=1
        else:
            traffic_signals[i]=0

        # Put hotspot information
        # We find the number of users for each link by finding the num of users for the nodes that touch the link 
        # Then we get the maximum number of users 
        if (this_node_csv.loc[link_index_v,hotspot_column_name]==1)|(this_node_csv.loc[link_index_u,hotspot_column_name]==1):
            hotspots[i]=1
        num_of_users_in_node_v = this_node_csv.loc[link_index_v,number_of_hotspot_users_column_name]
        num_of_users_in_node_u = this_node_csv.loc[link_index_u,number_of_hotspot_users_column_name]
        max_hotspot_users = np.max(np.array([num_of_users_in_node_v,num_of_users_in_node_u]))
        hotspot_users[i]=max_hotspot_users

    df_copy['Closest Poi Density_with distance decay']=link_csv_POIdata
    df_copy['Closest Poi Density_with distance decay'] = df_copy['Closest Poi Density_with distance decay'].fillna(0)
    df_copy['Traffic signals']=traffic_signals

    # These two columns are used so that we can include existing physiological data for this city
    df_copy['Hotspot']=hotspots
    df_copy['Hotspot users']=hotspot_users
    return(df_copy)

#### Add a column representing the time needed to walk on each link (street segment) from start to end 

In [7]:
def add_time_attribute(this_df,this_walking_speed):
    travel_speed = this_walking_speed
    #transform km per hour to m per minute
    meters_per_minute = travel_speed * 1000 / 60 
    this_df['time'] = 0
    this_df['time'] = this_df['length'] / meters_per_minute
    return(this_df)

#### make a df with selected columns for route optimization

In [8]:
# A definition to create the df with selected attributes
def create_csv_with_selected_attributes_for_optimisation(this_df):
    new_df = pd.DataFrame(this_df,columns=['u','v','traffic', 'length','Closest Poi Density_with distance decay', 'Traffic signals','time','Hotspot', 'Hotspot users'])
    new_df.loc[new_df[new_df['Closest Poi Density_with distance decay']==0].index,'Closest Poi Density_with distance decay']=1
    return(new_df)

#### Data transformation: Transform the selected attributes to fit in the same scale (0.1-1)

In [9]:
def transform_attributes(this_df, attribute_name, scaled_attribute_name):
    attribute_data = this_df[attribute_name]
    attribute_arr = np.reshape(np.array(attribute_data), (-1, 1))
    this_scaler = MinMaxScaler(copy=True, feature_range=(0.1, 1))
    this_scaler.fit(attribute_arr)
    attribute_arr = this_scaler.transform(attribute_arr)
    this_df[scaled_attribute_name]=attribute_arr
            

#### Put modifiers for each attribute 
If we want all the criteria to have the same importance, we can set the modifier value to 1 for all of them

In [10]:
def put_modifiers_in_the_attributes(this_df, length_m,traffic_m,density_m,traffic_signal_m,hotspot_m):
    # Create one column that sums the attributes for each link, after applying the modifiers
    this_df['Combined attributes'] = length_m*this_df['scaled Length']+traffic_m*this_df['scaled Traffic']+density_m*this_df['scaled Density'] + traffic_signal_m*this_df['scaled Signals'] 
    this_df['Combined attributes']=this_df['Combined attributes'].astype('int32')
    # Create another column which includes all other attributes and the hotspot attribute as well (in case we want to use this in the optimisation)
    
    this_df['Combined attributes and Hotspots']=length_m*this_df['scaled Length']+traffic_m*this_df['scaled Traffic']+density_m*this_df['scaled Density'] + traffic_signal_m*this_df['scaled Signals'] + hotspot_m*this_df['scaled Hotspot']
    this_df['Combined attributes and Hotspots']=this_df['Combined attributes and Hotspots'].astype('int32')

In [11]:
def calculate_seconds(speed, dist):
    tm = dist/(speed*1000)
    tm = tm*60*60
    return(tm)

##  ------  DEFINITIONS FOR EACH SCENARIO ------

#### DEFINITIONS FOR SCENARIO A
Find the shortest path based on all criteria (or only selected ones) 

In [12]:
def find_shortest_path_ScenarioA(this_graph, start, destination, attribute):
    shortest_path = nx.shortest_path(this_graph, source=start, target=destination, weight=attribute, method='dijkstra')
    # This returns a list of nodes from the node_csv, corresponding to the shortest path according to the 
    # selected criterion
    return(shortest_path)

#### DEFINITIONS FOR SCENARIO D 
(The scenario that includes isochrones)

#### Create isochrones for a range of trip duration times 

In [13]:
def create_isochrones(this_graph, trip_time_list,this_node,visualise_isochrones_on_map,
                     length_m,traffic_m,density_m,traffic_signal_m,hotspot_m):
    
    #The isochrone tutorial was found here:
    #https://github.com/gboeing/osmnx-examples/blob/master/notebooks/13-isolines-isochrones.ipynb
    
    # We will put a ranking in each node and edge, according to time needed to reach it from our central node
    #Also, create a dict for visualisation, containing one colour for each node based on the time needed to reach it from the target node
    
    edge_isochrones = {}
    node_colors = {}
    time_ranks = {}
    iso_colors = ox.plot.get_colors(n=len(trip_time_list), cmap='plasma', start=0, return_hex=True)
    for edge in new_G.edges:
        edge_isochrones[edge]=1000

    for trip_time, color in zip(sorted(trip_time_list, reverse=True), iso_colors):
        subgraph = nx.ego_graph(this_graph, this_node, radius=trip_time, distance='time')
        # This is the necessary part for computing the time-based penalty for the edge attributes
        for edge in subgraph.edges():
            if edge in this_graph.edges:
                edge_isochrones[edge]=trip_time
        # This is only needed for visualisation
        if visualise_isochrones_on_map==True:
            for node in subgraph.nodes():
                node_colors[node] = str(color)
                time_ranks[node] = trip_time

    #Create a copy of the graph and add the time needed to reach each edge as an edge weight
    new_graph = this_graph.copy()
    nx.set_edge_attributes(new_graph, edge_isochrones, 'Edge isochrones')
    
    # Get the rest of the attributes 
    scaled_density_dict = nx.get_edge_attributes(new_graph,'scaled Density')
    scaled_traffic_dict = nx.get_edge_attributes(new_graph,'scaled Traffic')
    scaled_signal_dict = nx.get_edge_attributes(new_graph,'scaled Signals')
    scaled_length_dict = nx.get_edge_attributes(new_graph,'scaled Length')
    hotspot_dict = nx.get_edge_attributes(new_graph,'scaled Hotspot')
    iso_dict = nx.get_edge_attributes(new_graph,'Edge isochrones')
    
    #Create a new attribute that combines all other attributes and adds a multiplier based on time
    #The multiplier penalizes more the points that have large density etc. as time increases
    
    time_corrected_combined_criterion = {}
    time_corrected_combined_with_hotspots_criterion = {}
    for k in list(scaled_signal_dict.keys()):

        m = np.power(iso_dict[k],2)
        # In the other definition ('put_modifiers_in_the_attributes') we put only the modifiers 
        # Now we also include another modifier (m) based on the isochrone values
        
        # This is for D without hotspot inclusion
        combined = length_m*scaled_length_dict[k] + traffic_signal_m*scaled_signal_dict[k]*m + density_m*scaled_density_dict[k]*m + traffic_m*scaled_traffic_dict[k]*m
        
        # This is for scenario B+D, where we also include the hotspots in the computation
        combined_with_hotspots = length_m*scaled_length_dict[k] + traffic_signal_m*scaled_signal_dict[k]*m + density_m*scaled_density_dict[k]*m + traffic_m*scaled_traffic_dict[k]*m + hotspot_m*hotspot_dict[k]*m
        
        
        time_corrected_combined_criterion[k]=combined
        time_corrected_combined_with_hotspots_criterion[k]=combined_with_hotspots
    nx.set_edge_attributes(new_graph, time_corrected_combined_criterion, 'Time-corrected combined attributes')
    nx.set_edge_attributes(new_graph, time_corrected_combined_with_hotspots_criterion, 'Time-corrected combined attributes with hotspots')
    
    
    # return the copy of the graph with the isochrone attributes
    return([new_graph,node_colors,time_ranks])

#### Calculate time needed to walk each path

In [14]:
# this path: the route for which we want to calculate time
def calculate_route_time(this_path, walking_speed):
    this_route_time = 0
    
    for i in range(0, len(this_path)-1):
        start = this_path[i]
        end = this_path[i+1]
        if len(link_csv_for_nx[(link_csv_for_nx['u']==start)&(link_csv_for_nx['v']==end)])!=0:
            this_df = link_csv_for_nx[(link_csv_for_nx['u']==start)&(link_csv_for_nx['v']==end)]
            this_length = this_df['length']
            this_route_time = this_route_time+calculate_seconds(walking_speed, this_length.values[0])

    print('Route time: ' + str(int(this_route_time/60)) + ' minutes')
    # The function returns route time in minutes
    return(int(this_route_time/60))

## ------ Definitions for visualisation ------

#### If we have only 1 created path: Visualise the created path 

In [15]:
def visualise_shortest_path(this_graph, this_title, this_path, node_colour,
                           render_combined_stressors,node_df_for_stressors):
    
    this_map = ox.folium.plot_graph_folium(this_graph, graph_map=None, popup_attribute=None, tiles='cartodbpositron', zoom=1, location = [-33.8756785, 151.22353900000002],fit_bounds=True, edge_color='salmon', edge_width=0.5, edge_opacity=0.5)
    this_map.save( this_title + ".html" )


    cnt=0

    for item in node_list:

        this_pt = (node_csv.loc[item, 'y'],node_csv.loc[item, 'x'])
        this_radius = 0.5
        this_opacity=0.3

        this_index = 'non'
        if item in this_path:
            this_colour = node_colour
            this_radius = 4
            this_opacity=0.9
            this_index = this_path.index(item)

        if this_index!='non':
            folium.CircleMarker([this_pt[0],this_pt[1]], color=this_colour, fill=True, radius=this_radius, 
                                opacity=this_opacity,fill_opacity = this_opacity, fill_color=this_colour,
                       popup = str(item)+'_'+str(this_index)).add_to(this_map)

    #-----RENDER ALL THE STRESSORS-----
    if render_combined_stressors == True:
        steps = 10
        colormap = branca.colormap.linear.YlOrRd_09.scale(0.0, 1.5).to_step(steps)
        gradient_map=defaultdict(dict)
        for i in range(steps):
            gradient_map[1/steps*i] = colormap.rgb_hex_str(1/steps*i)
        gmap=gradient_map


        node_df_for_stressors['Traffic signal']=0
        node_df_for_stressors.loc[node_df_for_stressors[node_df_for_stressors['highway']=='traffic_signals'].index, 'Traffic signal']=1
        
        this_df = node_df_for_stressors[(node_df_for_stressors['Closest Poi Density_with distance decay']>15)|(node_df_for_stressors['traffic']>2)|(node_df_for_stressors['Traffic signal']==1)].copy()
        # If we want to visualise only the hotspots of one stressor, delete the others, like in the following line (which should visualise only the POI density)
        # this_df = node_df_for_stressors[node_df_for_stressors['Closest Poi Density_with distance decay']>15].copy()
        this_df['Combined stressors']=1
        HeatMap(data=this_df[['y', 'x', 'Combined stressors']].groupby(['y', 'x']).sum().reset_index().values.tolist(),gradient=gmap, radius=7, max_zoom=13).add_to(this_map)



    this_map.save(this_title + ".html" )
    return(this_map)
    


#### Create a map to compare the generated shortest paths (if we have created many paths)

In [16]:
# If we have generated many different paths (according to different criteria) for the same starting and ending point,
# we can visualise them for comparison using this definition 
def create_map_for_comparison_of_generated_paths(this_graph, this_title, path_list, colour_list, criteria_list,
                                                render_combined_stressors,node_df_for_stressors):
    
    this_map = ox.folium.plot_graph_folium(this_graph, graph_map=None, popup_attribute=None, tiles='cartodbpositron', zoom=1, location = [-33.8756785, 151.22353900000002],fit_bounds=True, edge_color='salmon', edge_width=0.5, edge_opacity=0.5)
    #this_paths_map.save("Compare the generated shortest paths.html" )
    this_map.save( this_title + ".html" )

    cnt=0
    for item in node_list:
        this_pt = (node_csv.loc[item, 'y'],node_csv.loc[item, 'x'])

        #this_color='slategrey'
        #this_radius = 0.5
        #this_opacity=0.3

        for i in range (len(path_list)):
            this_path = path_list[i]
            if item in this_path:
                this_color = colour_list[i]
                this_criterion = criteria_list[i]
                this_radius = 2
                this_opacity = 0.5
                
       
                folium.CircleMarker([this_pt[0],this_pt[1]], color=this_color, 
                                fill_color=this_color, fill_opacity=this_opacity, fill=True, 
                                radius=this_radius, opacity=this_opacity,
                               popup = this_criterion).add_to(this_map)
        
    #-----RENDER ALL THE STRESSORS-----
    if render_combined_stressors == True:
        steps = 10
        colormap = branca.colormap.linear.YlOrRd_09.scale(0.0, 1.5).to_step(steps)
        gradient_map=defaultdict(dict)
        for i in range(steps):
            gradient_map[1/steps*i] = colormap.rgb_hex_str(1/steps*i)
        gmap=gradient_map


        node_df_for_stressors['Traffic signal']=0
        node_df_for_stressors.loc[node_df_for_stressors[node_df_for_stressors['highway']=='traffic_signals'].index, 'Traffic signal']=1
        
        this_df = node_df_for_stressors[(node_df_for_stressors['Closest Poi Density_with distance decay']>15)|(node_df_for_stressors['traffic']>2)|(node_df_for_stressors['Traffic signal']==1)].copy()
        # If we want to visualise only the hotspots of one stressor, delete the others, like in the following line (which should visualise only the POI density)
        # this_df = node_df_for_stressors[node_df_for_stressors['Closest Poi Density_with distance decay']>15].copy()
        this_df['Combined stressors']=1
        HeatMap(data=this_df[['y', 'x', 'Combined stressors']].groupby(['y', 'x']).sum().reset_index().values.tolist(),gradient=gmap, radius=7, max_zoom=13).add_to(this_map)


        '''
        #----ADD THIS PART TO RENDER THE EXISTING HOTSPOTS----

        steps = 10
        colormap2 = branca.colormap.linear.YlOrRd_09.scale(0.0, 1).to_step(steps)
        gradient_map2=defaultdict(dict)
        for i in range(steps):
            gradient_map2[1/steps*i] = colormap.rgb_hex_str(1/steps*i)
        gmap2=gradient_map2

        this_df = node_df_for_stressors[node_df_for_stressors['Hotspot']==1]
        HeatMap(data=this_df[['y', 'x', 'Hotspot']].groupby(['y', 'x']).sum().reset_index().values.tolist(),gradient=gmap2, radius=9, max_zoom=13).add_to(this_map)
        '''

    this_map.save(this_title + ".html" )
    return(this_map)

#### Visualise isochrones

In [17]:
def visualise_isochrones(this_graph,this_node,colour_list,time_rank_list):
      
    #----------Create a map to visualise the isochrones-----------
    this_map = ox.folium.plot_graph_folium(G, graph_map=None, popup_attribute=None, tiles='cartodbpositron', zoom=1, location = [-33.8756785, 151.22353900000002],fit_bounds=True, edge_color='salmon', edge_width=0.5, edge_opacity=0.5)
    this_map.save('isochrones for node X.html'.replace('X',str(this_node)))


    cnt=0
    for item in node_list:
        this_pt = (node_csv.loc[item, 'y'],node_csv.loc[item, 'x'])

        this_color='#605f6e'
        this_radius = 0.1
        this_opacity=0.3

        if item in this_graph.nodes():
            if item in colour_list:
                this_color=colour_list[item]
                this_radius = 20/time_rank_list[item]
                this_opacity=0.5


        folium.CircleMarker([this_pt[0],this_pt[1]], color=this_color, fill=True,
                                radius=this_radius, opacity=this_opacity, fill_opacity = this_opacity, 
                                fill_color=this_color).add_to(this_map)



    this_map.save('isochrones for node X.html'.replace('X',str(this_node)))

    return(this_map)
    #IFrame(src='./isochrones for node X.html'.replace('X',str(this_node)), width=1500, height=1500)

##  ------  HOW TO APPLY THE FUNCTIONS  ------

### 1. FOLLOW THE COMMON STEPS FOR ALL SCENARIOS 

In [ ]:
# Get the nodes within a 1800m area around the selected area (this is to help us with the visualisations 
# when we have a very large spatial database; later we will only visualise the points within the 1800m boundary
# so that the created map is not very heavy)
# We can increase the distance if it is needed, for larger routes
# Put here the central point of the area of the analysis 
c_point = (-33.882882592547034, 151.2063073174587)
my_area_graph = ox.graph_from_point((c_point[0],c_point[1]),dist=1800)

G = my_area_graph

In [ ]:
#----The following part was used during testing the algorithm for selecting pairs of nodes 
# It creates a clickable map where each node can be clicked to see its ID 
# Activate if needed
'''
for_node_selection = ox.folium.plot_graph_folium(G, graph_map=None, popup_attribute=None, tiles='cartodbpositron', zoom=1, location = [-33.8756785, 151.22353900000002],fit_bounds=True, edge_color='salmon', edge_width=0.5, edge_opacity=0.5)
for_node_selection.save("for node selection.html" )

cnt=0
for item in node_list:
    this_pt = (node_csv.loc[item, 'y'],node_csv.loc[item, 'x'])
    this_color='slategrey'
    this_radius = 0.5
    this_opacity=0.3

    folium.CircleMarker([this_pt[0],this_pt[1]], color=this_color, 
                            fill_color=this_color, fill_opacity=this_opacity, fill=True, 
                            radius=this_radius, opacity=this_opacity,
                   popup = str(item)).add_to(for_node_selection)
    
for_node_selection.save("for node selection.html" )
from IPython.display import IFrame

IFrame(src='./for node selection.html', width=1500, height=1500)
'''

In [28]:
# ----IMPORT THE SPATIAL DATABASE---------
# Put here the path where the file with the hotspots from Component 2 was saved 
# (after applying the methods of the file 'Cluster analysis').
# This file should be in the spatial database and it should also contain the network nodes (with the hotspots of physiological responses, if available)
hotspot_path = r'C:\Users\demdr\Desktop\Testing the thesis functions\Project data\Spatial database\Urban network analysis_nodes_with_hotspot.csv'
# Put here the path of the file containing the network links. This should be in the spatial database 
# created after applying the file 'Building the spatial database --Step 4-POI and OSM data fusion' from Component 1
link_path = r'C:\Users\demdr\Desktop\Testing the thesis functions\Project data\Spatial database\Urban network analysis_links.csv'
# My old link path:  
#r'D:\UTS - Backup 30 Aug 2020\Data\Personal data-first tests\Personal data-first tests\Sydney urban network analysis_links.csv'
node_csv = pd.read_csv(hotspot_path)
link_csv = pd.read_csv(link_path)
# The index of the 'node_csv' file should be the node index (a unique ID used in OSM data for representation of each node)
node_csv = node_csv.set_index('Unnamed: 0.1')

#Create a list that contains only the nodes within the area around the Central Station (to render the map more efficiently)
#This point is not important for computation, only for visualization in the following map
node_list = []
for item in G.nodes():
    if item in node_csv.index:
        node_list.append(item)



# ----USE DEFINITION 'calculate_POI_density_for_each_link'
hotspot_column_name = 'Hotspot_Sum of EDR amplitudes_change' # Put another name here if needed
# This column contains information regarding if the node is close to a hotspot of physiological responses or not 
# This column was created from the script designed for Component 2 (search the line "#SAVE THE SPATIAL DATABASE WITH THE HOTSPOT INFO")
# The appropriate names following the column names used in the other script are the following: 'Hotspot_Sum of EDR amplitudes', 'Hotspot_Sum of EDR amplitudes_change'
# Instead of the name 'Hotspot_Sum of EDR amplitudes_change' here, use the name Hotspot_Sum of EDR amplitudes' or 'Hotspot_Sum of EDR amplitudes_change' according to the 
# goal of the analysis
num_of_users_col_name = 'Users in hotspot_Sum of EDR amplitudes_change' #Also modify this column name if needed. 
# This should be the column that stores information regarding the number of users in each hotspot



# If there is no data regarding hotspots of physiological responses, add 2 empty columns anyway
if hotspot_column_name not in node_csv.columns:
    print('No data for hotspots of physiological responses')
    node_csv[hotspot_column_name]=0
    node_csv[num_of_users_col_name]=0
    
link_csv = calculate_POI_density_for_each_link(link_csv,node_csv,'Hotspot', 'Users in hotspot')



# ----- ADD TIME ATTRIBUTE ----
# Use the definition to add the time attribute
# we put 4.5 km/h as the walking speed but this can change
link_csv = add_time_attribute(link_csv, 4.5)




# -----make a df with selected columns for route optimization------
link_csv_for_nx = create_csv_with_selected_attributes_for_optimisation(link_csv)
# Add this if necessary for keeping only links (street segments) below/above a certain length
link_csv_for_nx=link_csv_for_nx[link_csv_for_nx['length']<500]
# We can also filter it according to hotspot users and keep only hotspots with many users (replace 0 with any other 
# desired number of users to do this)
desired_number_of_users = 0
link_csv_for_nx=link_csv_for_nx[link_csv_for_nx['Hotspot users']>=desired_number_of_users]


# -----DATA TRANSFORMATION--------
# Apply the definition for transformation of attributes for a list of selected parameters 
# Any other parameters (i.e. temperature, slope) should be added here in the same way in the future 
# if we wish to add them in the algorithm (after adding them to the spatial database)
# We include a column for hotspots and hotspot users, even if it is filled with zeros
attributes_for_transformation = ['Closest Poi Density_with distance decay','traffic','length','Traffic signals',
                                'Hotspot']
new_names = ['scaled Density','scaled Traffic','scaled Length','scaled Signals',
             'scaled Hotspot']
for i in range(len(attributes_for_transformation)):
    this_attribute = attributes_for_transformation[i]
    new_col_name = new_names[i]
    transform_attributes(link_csv_for_nx,this_attribute, new_col_name)



# ------PUT MODIFIERS-------
# Apply definition for putting modifiers in the attributes:
# The values of the modifiers determine the relative influence of each attribute 
# Now, all the modifiers have the same influence apart from length, which has a higher penalty 
# Therefore, length will be prioritised, but the other attributes will also be taken into account
# To remove entirely some attributes, set their value to 0 (i.e. if we want only traffic and density, we can set the other values to 0)
length_modifier = 30
traffic_modifier = 20
density_modifier = 20
traffic_signal_modifier = 20
hotspot_modifier = 20

put_modifiers_in_the_attributes(link_csv_for_nx, length_modifier,traffic_modifier,density_modifier,
                               traffic_signal_modifier,hotspot_modifier)


#Create a df which contains only the nodes within the 1800m radius boundaries identified in the beginning
# (we only use this for visualisation purposes)
node_in_boundaries = pd.DataFrame()
for i in node_list:
    if i in node_csv.index:
        node_in_boundaries = node_in_boundaries.append(node_csv.loc[i])


# Construct a new graph from the link csv 
# The new graph has only the selected attributes (and the columns with the combined attributes)
new_G = nx.from_pandas_edgelist(link_csv_for_nx, source='u',target='v', edge_attr=['scaled Traffic','scaled Length','scaled Density', 'scaled Signals','Combined attributes','Combined attributes and Hotspots','scaled Hotspot','time'])




No data for hotspots of physiological responses


### 2. ADD THE STARTING AND ENDING POINT

In [30]:
# Select 2 nodes as the starting and ending point
# We could include here a small function for putting a GPS point as input and finding the closest node 

end_node = 1826240755
initial_node = 25921196

### 3. TEST THE CREATED SCENARIOS
Test one or more of the scenarios A, B or C 


#### Test scenario A

In [31]:
# Examples for finding the shortest path between the nodes 'initial_node' and 'end_node', based on different attributes:


# Find shortest path based on length
s_p_length = find_shortest_path_ScenarioA(new_G, initial_node, end_node, 'scaled Length')
# Find shortest path based on traffic
s_p_traffic = find_shortest_path_ScenarioA(new_G, initial_node, end_node, 'scaled Traffic')
# Find shortest path based on density
s_p_density = find_shortest_path_ScenarioA(new_G, initial_node, end_node, 'scaled Density')
# Find shortest path based on signals
s_p_signal = find_shortest_path_ScenarioA(new_G, initial_node, end_node, 'scaled Signals')
# Find shortest path based on all the criteria together
s_p_alltogether = find_shortest_path_ScenarioA(new_G, initial_node, end_node, 'Combined attributes')

# Find the route time: Put here the name of the path and the desired walking speed (in km/h)
route_time_for_length_based_path = calculate_route_time(s_p_length,4.5)
# Do the same for the other paths


Route time: 26 minutes


#### 3b. Test scenario B

In [32]:
# Find shortest path based on all together + Hotspots
s_p_alltogether_and_hotspots = find_shortest_path_ScenarioA(new_G, initial_node, end_node, 'Combined attributes and Hotspots')
#Find route time (for walking with 4.5km speed)
route_time_for_combined_stressors_and_hotspots = calculate_route_time(s_p_alltogether_and_hotspots,4.5)


Route time: 33 minutes


#### Test Scenario D

In [33]:
# The step_for_isochrones is 5 minutes (so all the nodes that are within 5 minutes from the start will have the same colour;
# the nodes that are more than 5 and within 10 minutes from the start will have another colour, and so on
step_for_isochrones = 5 
# Increase this value if needed
end_time = 70
trip_times = list(np.arange(step_for_isochrones,end_time,step_for_isochrones))


# Put True here to visualise the isochrones for this pair of nodes, False to exclude the visualisation
create_isochrones_for_a_node = create_isochrones(new_G, trip_times,initial_node, True,
                                                length_modifier,traffic_modifier,density_modifier,
                                                 traffic_signal_modifier,hotspot_modifier)

# Use the extracted data to visualise the isochrones 
isochrone_graph = create_isochrones_for_a_node[0]
node_iso_clrs = create_isochrones_for_a_node[1]
node_iso_timeranks = create_isochrones_for_a_node[2]

# Use the new graph to find the shortest path, with all the criteria and time-penalty
s_p_timecorrected=nx.shortest_path(isochrone_graph, source=initial_node, target=end_node, weight='Time-corrected combined attributes', method='dijkstra')
#Find route time (for walking with 4.5km speed)
route_time_for_combined_stressors_and_time_penalty = calculate_route_time(s_p_timecorrected,4.5)


isochrone_map = visualise_isochrones(new_G,initial_node, node_iso_clrs,node_iso_timeranks)
#This can be activated to display the isochrone map
#IFrame(src='./isochrones for node X.html'.replace('X',str(initial_node)), width=1500, height=1500)

Route time: 33 minutes


#### Test scenario B+D

In [34]:
# The step is 5 minutes (so all the nodes that are within 5 minutes from the start will have the same colour;
# the nodes that are more than 5 and within 10 minutes from the start will have another colour, and so on
step_for_isochrones = 5 
# Increase this value if needed
end_time = 70
trip_times = list(np.arange(step_for_isochrones,end_time,step_for_isochrones))


# Put True here to visualise the isochrones for this pair of nodes, False to exclude the visualisation
create_isochrones_for_a_node = create_isochrones(new_G, trip_times,initial_node, True,
                                                length_modifier,traffic_modifier,density_modifier,
                                                 traffic_signal_modifier,hotspot_modifier)
# Use the extracted data to visualise the isochrones 
isochrone_graph = create_isochrones_for_a_node[0]
node_iso_clrs = create_isochrones_for_a_node[1]
node_iso_timeranks = create_isochrones_for_a_node[2]

# Use the new graph to find the shortest path, with all the criteria AND time-penalty AND hotspots
s_p_timecorrected_with_hotspots=nx.shortest_path(isochrone_graph, source=initial_node, target=end_node, weight='Time-corrected combined attributes with hotspots', method='dijkstra')

#Find route time (for walking with 4.5km speed)
route_time_for_combined_stressors_and_time_penalty_and_hotspots = calculate_route_time(s_p_timecorrected_with_hotspots,4.5)

isochrone_map = visualise_isochrones(new_G,initial_node, node_iso_clrs,node_iso_timeranks)
#Same as above; This can be activated to display the isochrone map
#IFrame(src='./isochrones for node X.html'.replace('X',str(initial_node)), width=1500, height=1500)

Route time: 39 minutes


#### 4. Visualise the created path(s)

In [25]:
# Example of using the definition for visualising one path 

# Display the created map: visualise shortest path according to length
# We can use this to visualise paths for the following: 
# s_p_traffic,s_p_signal,s_p_density,s_p_alltogether, s_p_alltogether_and_hotspots, 
# s_p_timecorrected, s_p_timecorrected_with_hotspots

title = 'shortest path - length'
display_title = './' + title + '.html'
shortest_path_based_on_length = visualise_shortest_path(G, title, s_p_length, 'tomato',
                                                       True,node_in_boundaries)
IFrame(src=display_title, width=1500, height=1500)

In [26]:
# Use the following code for visual comparison of the generated shortest paths, if we have 
# many paths to compare (generated by using different criteria in scenario A, or using other scenarios)
title = 'Compare the generated shortest paths2'
display_title = './' + title + '.html'
# Put here the paths that we want to compare
paths = [s_p_length,s_p_traffic,s_p_density,s_p_signal,s_p_alltogether,s_p_alltogether_and_hotspots]
colours = ['tomato','teal','darkseagreen','greenyellow','crimson','black']
criteria = ['Length-based optimisation','Traffic-based optimisation','Density-based optimisation','Traffic signal-based optimisation',
                'All criteria-based optimisation','All criteria-based optimisation plus hotspots']

compare_paths = create_map_for_comparison_of_generated_paths(G, title, paths,colours,criteria,
                                                            True,node_in_boundaries)
IFrame(src=display_title, width=1500, height=1500)